Sources: https://stackoverflow.com/questions/66884270/text-data-clustering-with-python

In [ ]:
!pip install distance
!pip install texthero
!pip install fuzzywuzzy

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import AffinityPropagation
import distance
import texthero as hero
from fuzzywuzzy import process
from operator import itemgetter

df = pd.DataFrame([
              'Italy', 
              'Italie', 
              'Italia', 
              'Italy',
              'Russia',
              'Russie',
              'Russia',
              'Russi',
])
df

In [ ]:
#IT MAKES A HUGE DIFFERENCE IF WE HAVE OR NOT HAVE LOWERCASE
#it uses Levenshtein distance between words
def string_matching(list1, damping, Verbose=False):
  #print(damping)
  words = np.asarray(list1) #So that indexing with a list will work
  lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])
  affprop = AffinityPropagation(affinity="precomputed", damping=damping)
  affprop.fit(lev_similarity)
    
  cluster_list = list()
  for cluster_id in np.unique(affprop.labels_):
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    if Verbose == True:
      print(" - *%s:* %s" % (exemplar, cluster_str))
    cluster_list.append(exemplar)
  #print(cluster_list)
  return cluster_list

#string_matching(try_list, 0.6)
def extract_classes(df, col, top_n, damping, Verbose=False):
  #try_list = hero.clean(df['ProprietaEdificio'].dropna()[2:]).unique()
  #try_list
  top50_clean = list(hero.clean(df[col].value_counts())[0:top_n].dropna().reset_index()['index'])
  top50_clusters = string_matching(top50_clean, damping)
  if Verbose == True:
    print('top_' + str(top_n) + ' cleaned:', '\t\t', top50_clean)
    print('top_' + str(top_n) + ' clusters:', '\t', top50_clusters)
    print('\n')
  return top50_clusters

def clean_df(df, col, choices, score_cutoff, Verbose=False):
  df2 = pd.DataFrame(df[col].dropna().apply(lambda x : process.extractBests(x, choices, score_cutoff=score_cutoff)))
  df2['original'] = df[col]
  df2 = df2[df2[col].apply(len) > 0]
  if Verbose == True:
    print('original:', df2[col].count())
    print('matched:', df[col].count())
    print('percentage:', df[col].count()/df2[col].count())
  def select_best(tuples):
    return max(tuples, key=itemgetter(1))[0]

  df2['best_match'] = df2[0].apply(select_best)
  df2.columns = ['matches', 'original', 'best_match']
  return df2

choices = extract_classes(df, 0, 7, 0.5, True)
choices = ['Russia', 'Italia']
cleaned_df = clean_df(df, 0, choices, 20, True)
cleaned_df